In [75]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv('spam_ham_dataset.csv')


In [76]:
from nltk.tokenize import RegexpTokenizer

def clean_str(string, reg = RegexpTokenizer(r'[a-z]+')):
    string = string.lower()
    tokens = reg.tokenize(string)
    return " ".join(tokens)

df['text_clean'] = df['text'].apply(lambda string: clean_str(string))
del df['text']
df = df[:2000]
df


,id,label,label_num,text_clean
0,605,ham,0,subject enron methanol meter this is a follow ...
1,2349,ham,0,subject hpl nom for january see attached file ...
2,3624,ham,0,subject neon retreat ho ho ho we re around to ...
3,4685,spam,1,subject photoshop windows office cheap main tr...
4,2030,ham,0,subject re indian springs this deal is to book...
...,...,...,...,...
1995,1216,ham,0,subject re saudi arabia daren i ll follow up o...
1996,1094,ham,0,subject july co owner s volumes july lst co ow...
1997,5049,spam,1,subject greatly improve your stamina i ve been...
1998,2467,ham,0,subject calpine daily gas nomination ricky a a...


In [73]:
n = [1,2,3,4,5]
n[-1]

5

In [29]:
# Convert a collection of text documents to a matrix of token counts
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(df.text_clean)

# Get the categories
y = df.label

# Split arrays or matrices into random train and test subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [30]:
# Find the best hyperparameter with GridSearchCV
# Exhaustive search over specified parameter values for an estimator.
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
parameters = {"alpha": [0.2,1,2,5,10], "fit_prior": [True, False]}

grid = GridSearchCV(MultinomialNB(), param_grid=parameters)
grid.fit(X_train,y_train)

# Create a DataFrame with the best Hyperparameters
pd.DataFrame(grid.cv_results_)[['params','mean_test_score']]\
                               .sort_values(by="mean_test_score", ascending=False)

# Create the model with the best hyperparameters
from sklearn.naive_bayes import MultinomialNB
alpha, fit_prior = grid.best_params_['alpha'], grid.best_params_['fit_prior']
model = MultinomialNB(alpha = alpha)

model.fit(X_train,y_train)
y_pred = model.predict(X_test)

from sklearn.metrics import classification_report, accuracy_score
print('Accuracy: ', round(accuracy_score(y_test,y_pred),3)*100)

Accuracy:  96.0


In [31]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         ham       0.98      0.97      0.97       306
        spam       0.90      0.94      0.92        94

    accuracy                           0.96       400
   macro avg       0.94      0.95      0.95       400
weighted avg       0.96      0.96      0.96       400

